# Tutorial #1: APIを用いたeLabFTWのノート作成

<b>Note:</b> ご不明な点がございましたら、lee.jemyung@naist.ac.jp にご連絡ください

## **Step 2:** APIを使っていろいろなテーブルのデータを入れてみましょう
#### データを保存するテーブルの自動検索、テーブルサイズの自動調整、データ追加及びファイルアップロードをAPIを通じて行うサンプルコード

#### Note: _Step 2_ で<span style='color: orange'>新しくお知らせする部分のタイトル</span>はオレンジ色で表示し、その他の部分では _Step 1_ と同じ内容となります。

## 前準備
###　「.env」ファイルの作成
「.env」ファイルの作成に関しては[こちら](https://github.com/naist-eln/eln/blob/main/API/eLabFTW_API_env.ipynb)をご覧ください。<br>
「.env」ファイルがすでに用意されている場合は、そのまま次に進んでください。

### <span style='color: orange'>更新内容の入力</span>
実験ノートにアップデートするテーブルデータを入力します。<br>
「template_id」: テンプレートのID (省略可)<br>
「template_title」: テンプレートのタイトル名<br>
「sample_id」: サンプルID<br>
「upload_file」: アップロードするファイルのパス(省略可)<br>
「exp_contents」: 入力するデータのdictionary（「テーブル名」: [（「タイトル」:「入力値」）] の構造になっています）　

In [ ]:
template_id = ''
template_title = 'Test_No1'
sample_id = '1001'

upload_file = r'./test_file/upload_test.txt'

# 更新する実験ノートの内容を入力
exp_contents = {
    '目的組成': [
        {
        'SampleID': sample_id,
        'TargetCompound': "1234",
        'Alpha': "1234",
        'ElementRatio': "123",
        'Method': "12345",
        'WC': "1235",
        'FWC': "1111"
        },
        {
        'SampleID': sample_id,
        'TargetCompound': "55",
        'Alpha': "25",
        'ElementRatio': "2",
        'Method': "150",
        'WC': "30",
        'FWC': "50"
        }
    ], 
    '実験条件': [
        {
        'SampleID': sample_id,
        'ReactionTemp': "1234",
        'ReactionTime': "1234"
        },
        {
        'SampleID': sample_id,
        'ReactionTemp': "55",
        'ReactionTime': "25"
        }
    ]
}

### コード駆動準備
ライブラリ、パッケージ、モジュールの呼び出しと設定

In [ ]:
import os
import re
import elabapi_python
from elabapi_python.rest import ApiException
import json
from requests.exceptions import HTTPError
import numpy as np
import pandas as pd
from pandas import json_normalize
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import codecs
import smbclient
from smb.SMBConnection import SMBConnection
from dotenv import load_dotenv
from urllib.parse import urlparse
import warnings
from urllib3.exceptions import InsecureRequestWarning
from difflib import get_close_matches

# APIクライアントの設定
load_dotenv()
configuration = elabapi_python.Configuration()
configuration.host = os.environ["HOST_URL"]
configuration.api_key['Authorization'] = os.environ["API_KEY"]
configuration.debug = False
configuration.verify_ssl = False

# SSL警告を無効化
warnings.simplefilter('ignore', InsecureRequestWarning)

# 各種のインスタンスを作成
api_client = elabapi_python.ApiClient(configuration)
api_client.set_default_header(header_name = 'Authorization', header_value = configuration.api_key['Authorization'] )
experimentsApi = elabapi_python.ExperimentsApi(api_client)
templatesApi = elabapi_python.ExperimentsTemplatesApi(api_client)
uploadsApi = elabapi_python.UploadsApi(api_client)
itemsApi = elabapi_python.ItemsApi(api_client)

## 実験ノート作成

### テンプレートリストの確認
現在登録されているテンプレートのリストを表示します。テンプレートのIDとTitleを確認することができます。

In [ ]:
# テンプレートIDとタイトルを取得
templates_data = []
errors = []

try:
    templates_list = templatesApi.read_experiments_templates()
    
    for template in templates_list:
        try:
            templates_data.append({
                'ID': template.id,
                'Title': template.title
            })
        except AttributeError as e:
            errors.append(f"AttributeError for template ID {template.id}: {e}")
        except Exception as e:
            errors.append(f"An error occurred for template ID {template.id}: {e}")

except ApiException as e:
    print(f"Exception when calling TemplatesApi->read_experiments_templates: {e}\n")
    if e.body:
        print(f"Error details: {e.body}\n")

# データフレームに変換して表示
templates_df = pd.DataFrame(templates_data)
templates_df[['ID']] = templates_df[['ID']].astype(str)
display(templates_df)

# エラーの内容を表示
if errors:
    print("Errors encountered:")
    for error in errors:
        print(error)

### テンプレートの割り当て
指定したIDまたはTitleからテンプレートを割り当て、IDとTitleを確定</br>
コード実行時に割り当てられたIDとTitleが表示される。

In [ ]:
detected_title = ""
try:
    if template_id and (templates_df['ID'] == template_id).any():
        detected_title = templates_df.loc[templates_df['ID'] == template_id, 'Title'].values[0]
        if not template_title:
            template_title = detected_title
    elif template_title:
        if (templates_df['Title'] == template_title).any():
            detected_title = template_title
        else:
            detected_title = get_close_matches(template_title, templates_df['Title'] , 3, 0.7)
            if detected_title:
                detected_title = detected_title[0]
            else:
                raise ValueError(f"Template title is not detected.")
        template_id = templates_df.loc[templates_df['Title'] == detected_title, 'ID'].values[0]
    else:
        raise ValueError("Please enter a valid template ID or Title, above.")
    
    print(f"Template title is assigned to \"{detected_title}\"" + ("." if detected_title == template_title else f", altered from \"{template_title}\"."))

    if template_id in templates_df['ID'].values:
        print(f"Template ID: {template_id}")
    else:
        raise ValueError(f"Template ID {template_id} not found in templates_df")
except ValueError as e:
    print("ValueError: %s\n" % e)

### テンプレートの取得
割り当てられたIDのテンプレートを取得

In [ ]:
# Prepare the body for creating an experiment
template = None
experiment_body = {
    "category_id": template_id,  # カテゴリIDを指定
}

try:
    if template_id in templates_df['ID'].values:
        template = templatesApi.get_experiment_template(template_id)
        print(f"Template (ID: {template_id}) is ready to use.")
    else:
        raise ValueError(f"Template ID {template_id} not found in templates dataframe.")
except ValueError as e:
    print("ValueError: %s\n" % e)

### テンプレートの確認
取得したテンプレートを確認

In [ ]:
try:    
    template_dict = template.to_dict()
    template_html = "<h2>Template Information</h2>"
    template_html += "<ul>"
    for key, value in template_dict.items():
        template_html += f"<li><strong>{key}:</strong> {value}</li>"
    template_html += "</ul>"
    display(HTML(template_html))
except ApiException as e:
    print(f"Exception when calling TemplatesApi->get_experiment_template: {e}\n")
    if e.body:
           print(f"Error details: {e.body}\n")
except Exception as e:
    print(f"An error occurred: {e}\n")

### 実験ノートの準備
テンプレートから「(Template title)_(Sample ID)」をタイトルにする実験ノートを作成してeLabFTWに登録する。</br>
正常に登録が完了したら、その結果を出力する。

In [ ]:
try:
    # テンプレートIDがデータフレームに存在するか確認
    if template_id not in templates_df['ID'].values:
        display(templates_df)
        print(template_id, " ", type(template_id))
        raise ValueError(f"Template ID {template_id} not found in templates_df")

    # テンプレートIDに対応するタイトルを取得
    template_title = templates_df.loc[templates_df['ID'] == template_id, 'Title'].values[0]
    
    # 新しいタイトルを作成
    new_title = f"{template_title}_{sample_id}"
    
    # 実験ノートの内容を設定
    experiment_body = {
        "category_id": str(template_id)  # カテゴリIDを指定
    }
   
    # Create an experiment
    api_response = experimentsApi.post_experiment(body=experiment_body)    
    print("Experiment created successfully: ")
    if api_response is None: print(f"{api_response} issue.")
    else: print(api_response)

    # 新しい実験ノートのIDを取得するため、最新の実験ノートを取得
    experiments_list = sorted(experimentsApi.read_experiments(), key=lambda x: x._date, reverse=True)
    new_experiment_id = experiments_list[0].id

    # 実験ノートのタイトルを更新
    update_body = {
        "title": new_title
    }
    update_response = experimentsApi.patch_experiment(new_experiment_id, body=update_body)
    print("Experiment title updated successfully:")
    print(update_response)

except ValueError as e:
    print("ValueError: %s\n" % e)
except ApiException as e:
    print("Exception when calling ExperimentsApi->post_experiment or patch_experiment: %s\n" % e)
    if e.body:
        print(f"Error details: {e.body}\n")
except Exception as e:
    print("An error occurred: %s\n" % e)

### 実験ノートリストの確認
作成した全実験ノートを出力する。

In [ ]:
# 全実験ノートの確認
try:
    # デフォルト設定で実験ノートを取得
    print("デフォルト設定で実験ノートを取得:")
    experimentsList = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experimentsList)}")
    
    # 必要なフィールドを抽出してリストに格納
    exp_data = []
    for experiment in experimentsList:
        exp_dict = experiment.to_dict()
        exp_data.append({
            '_date': exp_dict.get('_date'),
            'category_title': exp_dict.get('category_title'),
            'category': exp_dict.get('category'),
            'fullname': exp_dict.get('fullname'),
            'id': exp_dict.get('id'),
            'tags': exp_dict.get('tags'),
            'tags_id': exp_dict.get('tags_id'),
            'up_item_id': exp_dict.get('up_item_id')
        })

    # データフレームに変換
    experiments_df = pd.DataFrame(exp_data)
    experiments_df['_date'] = pd.to_datetime(experiments_df['_date'])   # _date列をdatetime型に変換
    experiments_df = experiments_df.sort_values(by='_date', ascending=False)    # 作成日が新しい順にソート
    print(experiments_df)
    
except ApiException as e:
    print("ExperimentsApiを呼び出す際の例外: %s\n" % e)

### 最新の実験ノートを取得
実験ノートリストから最新の実験ノートを取得する。</br>
取得した実験ノートのIDが表示されます。

In [ ]:
try:
    experiments_list = experimentsApi.read_experiments()
    print(f"Number of experiments: {len(experiments_list)}")
    
    # 最も新しい実験ノートを取得
    if experiments_list:
        experiments_list = sorted(experiments_list, key=lambda x: x._date, reverse=True)  # 作成日でソート
        experiment_id = experiments_list[0].id  # 最新の実験ノートのIDを取得
        print(f"Latest experiment ID: {experiment_id}")        
    else:
        print("No experiments found.")
        exit()

except ApiException as e:
    print(f"Exception when calling ExperimentsApi->read_experiments: {e}\n")
    exit()

## <span style='color: orange'>実験ノートの内容更新</span>

### <span style='color: orange'>ノート更新の準備</span>
テーブルの基本スタイルを設定し、テーブルに列を追加する関数とデータから特殊文字を書き出し、テキストだけを抽出する関数を定義

In [ ]:
# Define the template's default line color and border style.
default_style = {
    'border-color' : '#95a5a6',
    'border-style' : 'solid'
}

# Add 'n_row' empty row(s) in the transferred 'table' of 'soup', 
# reflecting the row('tr_style') and cell('td_style') styles.
def addEmptyRow(table, soup, n_row = 1, tr_style = '', td_style = ''):
    
    n_cell = len(table.tr.find_all('td'))
    
    for i in range(n_row):
        empt_tr = soup.new_tag('tr', style = tr_style) if tr_style else soup.new_tag('tr')
    
        for j in range(n_cell):
            nw_td = soup.new_tag('td', style = td_style)
            nw_td.string = ' '
            empt_tr.append(nw_td)
            
        table.append(empt_tr)
    
    return table

# Pass an empty soup of row, which has 'n_cell' cell(s) and 'tr_style' style.
# Warning: Currently, this function does not work perfectly. 
#          BeautifulSoup does not reflect changes in lower structures.
def getEmptyRowSoup(n_cell, soup, tr_style = []):
    st_items = [tr_st.split(':')[0] for tr_st in tr_style]
    df_st = {k: v for k, v in default_style.items() if k not in st_items}
    
    r_sty = ';'.join(tr_style) + ';' if tr_style else None
    c_sty = ''
    for k in df_st.keys(): 
        c_sty += f'{k}:{df_st[k]};'
        
    if tr_style:
        for st in tr_style: 
            c_sty += f'{st};'
    row = soup.new_tag('tr', style = r_sty) if r_sty else soup.new_tag('tr')
    
    empt_cels = [soup.new_tag('td', style = c_sty) for i in range(n_cell)]
    row.extend(empt_cels)

    return row

# This returns a body of empty row that has 'n_cell' cell(s).
def getEmptyRowBody(n_cell, tr_style):
    st_items = [tr_st.split(':')[0] for tr_st in tr_style]
    df_st = {k: v for k, v in default_style.items() if k not in st_items}
    
    r_bd = '<tr' + ((' style=\"' + ';'.join(tr_style) + ';\"') if tr_style else '') + '>'
    
    c_bd = '\n<td style=\"'
    for k in df_st.keys(): 
        c_bd += f'{k}:{df_st[k]};'
    if tr_style:
        for st in tr_style: 
            c_bd += f'{st};'
    c_bd += '\"> </td>'
    
    for i in range(n_cell): r_bd += c_bd
        
    r_bd += '\n</tr>'
    
    return r_bd

# Remove special characters from the string 'str'
# Can define more removing character(s) by adding it at 'char'.
def removeSpChar(str, char):
    try:
        rv_str = re.sub(r'[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', str)
        if char: rv_str = rv_str.lstrip(char).strip()
        return rv_str
    except Exception as e:
        print(f"An error occurred: {e}\n")

### <span style='color: orange'>実験ノートの内容を更新</span>
最新の実験ノートの詳細を取得して実験ノートの内容を更新する。<br>
'exp_contents' のデータをノートの各テーブルに追加します。<br>
次のようなプロセスでテーブルにデータを追加します。<br>
   1. 'exp_contents' dictionaryの'テーブル名'に該当するテーブルをノートで検索して、<br>
   2. 名前が一致するか似たようなテーブルに各データを追加します。<br>
   3. テーブルにデータを追加するとき、「exp_contents」内に定義された「タイトル」に該当するテーブルのColumnに「入力値」を追加します。<br>
    参考：「exp_contents」は「テーブル名」: [（「タイトル」:「入力値」）] のDictionaryになっています

In [ ]:
# 最新の実験ノートの詳細を取得
try:
    latest_experiment= experimentsApi.get_experiment(experiment_id)
    exp_body = latest_experiment.body
    exp_title = latest_experiment.title

    # BeautifulSoupを使ってHTMLを解析
    soup = BeautifulSoup(exp_body, 'html.parser')
    
    cnt_soup = soup.find_all(['h1','p'])
    cnt_dict = dict()
    for cs in cnt_soup:
        cst = removeSpChar(cs.text, 'TS')
        if cst:
            cnt_dict[cst] = cs

    for es in list(exp_contents.keys()):
        ec_list = exp_contents[es]
        n_exp = len(ec_list)
        
        # cnt_soup = soup.find(['h1','p'], text = re.compile('(.*)'+es+'(.*)'))   # legacy code
        if es in cnt_dict:
            cnt_soup = cnt_dict[es]
        else:
            candidate = get_close_matches(es, list(cnt_dict.keys()), 3, 0.8)
            candidate = candidate[0] if candidate else "None"
            raise ValueError(f"Column {es} not found in table: candidate = {candidate}")
        table = cnt_soup.find_next_sibling('table')
        
        if table:
            rows = table.find_all('tr')
            
            if len(rows) > 1:
                items = list(map(lambda x:x.text, rows[1].find_all('td')))     # Column titles (English ver.)
                n_items = len(items)

                chk_empt_row = [all([c.get_text(strip=True) in ['', None] for c in r.find_all('td')]) for r in rows]
                n_empt_row = np.count_nonzero(chk_empt_row)
                n_filled_row = len(rows) - n_empt_row
                
                # Add empty row(s) if it's insufficient
                if n_empt_row < n_exp:
                    tr_style = rows[2]['style'] if rows[2].attrs.get('style') else ''
                    td_style = rows[2].td['style']
                    
                    addEmptyRow(table, soup, n_exp - n_empt_row, tr_style, td_style)
                    rows = table.find_all('tr')
                    
                ## legacy code 
                # if n_empt_row < n_exp:
                #     tr_style = [rows[2]['style']] if (len(rows) > 2 and rows[2].attrs.get('style')) else []
                #     # rows.extend([getEmptyRowSoup(n_items, soup, tr_style) for i in range(n_exp - n_empt_row)])
                #     
                #     for i in range(n_exp - n_empt_row):
                #         rows.append(getEmptyRowSoup(n_items, soup, tr_style))
                
                # Reflect the contents
                for i, ec in enumerate(ec_list):               
                    cells = rows[i + n_filled_row].find_all('td')      # Experiment values         
                    for ec_title in ec.keys():
                        if ec_title in items:
                            cells[items.index(ec_title)].string = ec[ec_title]
                        else:
                            candidate = get_close_matches(ec_title, items, 3, 0.8)
                            candidate = candidate[0] if candidate else "None"
                            raise ValueError(f"Column {ec_title} not found in table: candidate = {candidate}")
            else:
                raise ValueError(f'Insufficient rows: size {len(rows)}')

    # 更新されたHTMLを文字列に戻す
    rev_exp_body = str(soup)

    # 新しい内容をもとに既存の内容を更新
    update_body = {
        "body": rev_exp_body,  # 更新されたHTML
        # "title": exp_title + " - 更新しました"  # タイトルを更新
    }

    # リクエスト内容を表示
    # print("Request body:")
    # print(json.dumps(update_body, ensure_ascii=False, indent=2))

    # 実験ノートの内容を更新
    update_response = experimentsApi.patch_experiment(experiment_id, body=update_body)
    print("Experiment updated successfully:")
    # print(update_response)
except ApiException as e:
    print(f"Exception when calling ExperimentsApi->patch_experiment: {e}\n")
    if e.body:
        print(f"Error details: {e.body}\n")
except Exception as e:
    print(f"An error occurred: {e}\n")

### <span style='color: orange'>ファイルのアップロード</span>
「upload_file」にあるファイルをノートにアップロードします。

In [ ]:
try:
    if upload_file:
        uploadsApi.post_upload('experiments', experiment_id, file=upload_file, comment='Uploaded with APIv2')
        print(f'File \"{upload_file}\" has uploaded to experiment #{experiment_id}.')
    else:
        print("No file to upload")

except ApiException as e:
    print("UploadsApiを呼び出す際の例外: %s\n" % e)